In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession 
import pyspark.sql.functions as F 
from pyspark.sql.types import *

In [2]:
table = StructType([
    StructField("searchDate", DateType(), False), 
    StructField("flightDate", DateType(), False),
    StructField("startingAirport", StringType(), False),
    StructField("destinationAirport", StringType(), False),
    StructField("travelDuration", StringType(), False),
    StructField("totalFare", FloatType(), False),
    StructField("totalTravelDistance", FloatType(), False),
    StructField("days_between_search_and_flight", IntegerType(), False)
])

In [3]:
spark = (SparkSession.builder.appName("Serving").getOrCreate())

df = spark.read.csv("hdfs://namenode:9000/project/data_clean", schema=table)


In [4]:
df.printSchema()

root
 |-- searchDate: date (nullable = true)
 |-- flightDate: date (nullable = true)
 |-- startingAirport: string (nullable = true)
 |-- destinationAirport: string (nullable = true)
 |-- travelDuration: string (nullable = true)
 |-- totalFare: float (nullable = true)
 |-- totalTravelDistance: float (nullable = true)
 |-- days_between_search_and_flight: integer (nullable = true)



In [5]:
df.select("startingAirport", "destinationAirport").distinct().show(1000)
cnt_one_flight = (df.select("startingAirport", "destinationAirport")
        .where((F.col("startingAirport") == "JFK") & (F.col("destinationAirport") == "ORD"))
        .count())
cnt_one_flight

+---------------+------------------+
|startingAirport|destinationAirport|
+---------------+------------------+
|            JFK|               ORD|
|            MIA|               DTW|
|            BOS|               JFK|
|            CLT|               MIA|
|            LAX|               DFW|
|            DTW|               PHL|
|            JFK|               MIA|
|            ATL|               LGA|
|            BOS|               DEN|
|            DEN|               CLT|
|            ATL|               ORD|
|            PHL|               ATL|
|            DFW|               ORD|
|            DEN|               SFO|
|            IAD|               JFK|
|            LAX|               MIA|
|            ATL|               CLT|
|            DFW|               DEN|
|            DFW|               IAD|
|            DTW|               DEN|
|            CLT|               DTW|
|            CLT|               IAD|
|            IAD|               LAX|
|            JFK|               SFO|
|

101881

In [6]:
## First algorithm 
# Get specific flight journey (from starting airport to destination airport)
def specific_flight(startingAirport, destinationAirport):
    cnt_one_flight = (df.select("startingAirport", "destinationAirport", "totalFare", "days_between_search_and_flight")
        .where((F.col("startingAirport") == startingAirport) & (F.col("destinationAirport") == destinationAirport)))
    return cnt_one_flight

# Get statistics for a specific flight journey 
def get_stats_for_flight(flight):
    flight_stats = (flight.groupBy("days_between_search_and_flight")
    .agg(F.min("totalFare").alias("MinFare"), F.round(F.mean("totalFare"), 2).alias("MeanFare")
        , F.max("totalFare").alias("MaxFare")).sort(F.col("days_between_search_and_flight").asc())    
    )
    flight_stats.show()

flight = specific_flight("CLT", "MIA")
get_stats_for_flight(flight)


## Second algorithm
# Get top 5 destinations from a starting location/airport
def get_top5_destinations_from(starting_location):
    valid_airports = df.select("startingAirport").distinct().rdd.flatMap(lambda x: x).collect()
    if starting_location not in valid_airports:
        print(f"{starting_location} is not a valid starting airport code.")
        return
    
    flights_from = (df.select("startingAirport", "destinationAirport").where(F.col("startingAirport") == starting_location))
    hot_dest = (flights_from.groupBy("destinationAirport").count().sort(F.col("count").desc()))
    hot_dest.show(5)

get_top5_destinations_from("OAK")

## Third algorithm
# Get all airportcodes available in the dataset
def get_airports():
    airports = (df.select("startingAirport").union(df.select("destinationAirport"))
                .distinct().rdd.flatMap(lambda x: x).collect())
    return airports

get_airports() 

+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             1|  182.1|   422.8| 2175.1|
|                             2|  182.1|  362.84| 2175.1|
|                             3|  182.1|  350.49| 2175.1|
|                             4|  182.1|  346.25| 2175.1|
|                             5|  182.1|  342.08| 1017.1|
|                             6|  171.1|  315.83| 1161.1|
|                             7|  171.1|  314.27| 1161.1|
|                             8|  171.1|  312.56| 1161.1|
|                             9|  171.1|  308.87|  677.1|
|                            10|  171.1|  309.82| 766.11|
|                            11|  171.1|  309.42| 766.11|
|                            12|  171.1|  308.98| 766.11|
|                            13|  171.1|  310.79| 766.11|
|                            14|  110.1|  305.05| 766.11|
|             

+------------------+-----+
|destinationAirport|count|
+------------------+-----+
|               LAX|24971|
|               DEN| 4822|
|               DTW|  568|
|               DFW|    8|
|               ATL|    8|
+------------------+-----+



['OAK',
 'LGA',
 'BOS',
 'EWR',
 'DEN',
 'IAD',
 'CLT',
 'MIA',
 'DFW',
 'SFO',
 'ATL',
 'ORD',
 'DTW',
 'LAX',
 'JFK',
 'PHL']

In [7]:
# Showing all combinations of starting and destination airports
num_flight = df.select("startingAirport", "destinationAirport").distinct()
num_flight_opt = num_flight.count()
print(f"Number of distinct flights: {num_flight_opt}")
num_flight.show(num_flight_opt)


Number of distinct flights: 206


+---------------+------------------+
|startingAirport|destinationAirport|
+---------------+------------------+
|            JFK|               ORD|
|            MIA|               DTW|
|            BOS|               JFK|
|            CLT|               MIA|
|            LAX|               DFW|
|            DTW|               PHL|
|            JFK|               MIA|
|            ATL|               LGA|
|            BOS|               DEN|
|            DEN|               CLT|
|            ATL|               ORD|
|            PHL|               ATL|
|            DFW|               ORD|
|            DEN|               SFO|
|            IAD|               JFK|
|            LAX|               MIA|
|            ATL|               CLT|
|            DFW|               DEN|
|            DFW|               IAD|
|            DTW|               DEN|
|            CLT|               DTW|
|            IAD|               LAX|
|            CLT|               IAD|
|            JFK|               SFO|
|

In [ ]:
# This took 40 minutes to run
import time

# Collect the distinct flights into a list of rows
distinct_flights = num_flight.collect()

# Iterate over the list and pass the columns to the specific_flight function
time_table = []

for flight in distinct_flights:
    start_time = time.time()

    startingAirport = flight['startingAirport']
    destinationAirport = flight['destinationAirport']
    flight_df = specific_flight(startingAirport, destinationAirport)
    get_stats_for_flight(flight_df)

    end_time = time.time()
    time_table.append(end_time - start_time)
print({end_time - start_time})

+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             1|  108.6|  255.77| 1686.6|
|                             2|   93.6|  232.71| 1686.6|
|                             3|   85.6|  231.73|  804.6|
|                             4|   78.6|  227.46| 1686.6|
|                             5|   78.6|  227.34| 764.61|
|                             6|   78.6|  218.41|  804.6|
|                             7|   78.6|  218.67|  804.6|
|                             8|   78.6|  216.39|  804.6|
|                             9|   78.6|  214.24|  734.6|
|                            10|   78.6|  212.03| 724.61|
|                            11|   78.6|  211.72| 724.61|
|                            12|   78.6|  211.78|  718.6|
|                            13|   78.6|  211.51| 678.61|
|                            14|   78.6|  207.25| 678.61|
|             

+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             1| 296.61|  378.99| 1684.6|
|                             2| 296.61|  358.13| 1684.6|
|                             3| 296.61|  353.63| 1098.6|
|                             4| 296.61|  350.23| 1318.6|
|                             5| 296.61|  349.49| 1318.6|
|                             6|  138.6|  277.84| 1098.6|
|                             7|  128.6|  276.33| 1098.6|
|                             8|  128.6|  280.63| 1364.6|
|                             9|  128.6|  277.21| 1684.6|
|                            10|  128.6|  276.59| 1684.6|
|                            11|  128.6|   274.9| 1318.6|
|                            12|  128.6|  274.61| 1318.6|
|                            13|  128.6|  275.24| 1318.6|
|                            14|  108.6|  244.71| 1318.6|
|             

+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             1|   93.6|  207.89| 1535.6|
|                             2|   54.6|  178.12| 1535.6|
|                             3|   54.6|  181.09| 1535.6|
|                             4|   48.6|  177.94| 1535.6|
|                             5|   48.6|  176.86| 1535.6|
|                             6|   48.6|  168.44| 1535.6|
|                             7|   48.6|  170.05| 1535.6|
|                             8|   48.6|  168.71| 1535.6|
|                             9|   48.6|  168.09| 1535.6|
|                            10|   48.6|  168.33| 1535.6|
|                            11|   48.6|  167.06| 1535.6|
|                            12|   48.6|  167.09| 1535.6|
|                            13|   48.6|  165.19| 1535.6|
|                            14|   48.6|   155.7| 1535.6|
|             

+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             1|  182.1|   422.8| 2175.1|
|                             2|  182.1|  362.84| 2175.1|
|                             3|  182.1|  350.49| 2175.1|
|                             4|  182.1|  346.25| 2175.1|
|                             5|  182.1|  342.08| 1017.1|
|                             6|  171.1|  315.83| 1161.1|
|                             7|  171.1|  314.27| 1161.1|
|                             8|  171.1|  312.56| 1161.1|
|                             9|  171.1|  308.87|  677.1|
|                            10|  171.1|  309.82| 766.11|
|                            11|  171.1|  309.42| 766.11|
|                            12|  171.1|  308.98| 766.11|
|                            13|  171.1|  310.79| 766.11|
|                            14|  110.1|  305.05| 766.11|
|             

+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             1| 118.98|   360.4| 2499.6|
|                             2|  73.98|  333.98| 2499.6|
|                             3|  73.98|  332.26| 2499.6|
|                             4|  73.98|  325.69| 2499.6|
|                             5|  45.98|  320.25| 1098.6|
|                             6|  45.98|  311.79| 1368.6|
|                             7|  45.98|   311.2| 1368.6|
|                             8|  45.98|  312.55| 1368.6|
|                             9|  45.98|  306.32| 1368.6|
|                            10|  45.98|  303.84| 1368.6|
|                            11|  45.98|  302.57| 1368.6|
|                            12|  45.98|  301.85|  943.6|
|                            13|  45.98|  302.84|  943.6|
|                            14|  45.98|  267.78|  943.6|
|             

...

+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             1|  278.6|  336.62|1197.61|
|                             2|  278.6|  324.86|1197.61|
|                             3|  278.6|  324.46|1197.61|
|                             4|  159.6|  323.16|  418.6|
|                             5|  159.6|  323.62|1197.61|
|                             6|  119.6|  278.77|1197.61|
|                             7|  119.6|  272.92|  688.6|
|                             8|  119.6|  273.67|  688.6|
|                             9|  116.6|  273.88|  358.6|
|                            10|  116.6|  272.84|  378.6|
|                            11|  116.6|  273.26|  378.6|
|                            12|  116.6|  274.28|  378.6|
|                            13|  116.6|  274.33|  378.6|
|                            14|   90.6|  170.78|  378.6|
|             

+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             5|  648.6|   723.6| 1048.6|
+------------------------------+-------+--------+-------+



+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             5|  648.6|   723.6| 1048.6|
+------------------------------+-------+--------+-------+



+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             5|  648.6|   723.6| 1048.6|
+------------------------------+-------+--------+-------+



+------------------------------+-------+--------+-------+
|days_between_search_and_flight|MinFare|MeanFare|MaxFare|
+------------------------------+-------+--------+-------+
|                             5|  648.6|   723.6| 1048.6|
+------------------------------+-------+--------+-------+

{11.534762144088745}
